In [ ]:
import time
import warnings

from sklearn import datasets, cluster, mixture
from sklearn.decomposition import PCA

import random
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from scipy.cluster.hierarchy import dendrogram

from gmplabtools.pamm.lib.dimensionality import DataSampler
from gmplabtools.pamm.lib.tools import GMMPredict
from gmplabtools.pamm.pamm_commander import PammCommander
from gmplabtools.pamm.lib.clustering_tools import calculate_adjacency, adjancency_dendrogram
from gmplabtools.pamm.lib.transition_rates import ClusterRates

# Main

## 1) Inputs

In [ ]:
# directory of stored pca files
dir_ = '/mydir'

# individuals pca sets trained from the whole set
SYS1pca = np.loadtxt(dir_+'/bylayer273K.pca')
SYS2pca = np.loadtxt(dir_+'/bylayer293K.pca')
SYS3pca = np.loadtxt(dir_+'/bylayer323K.pca')
SYS4pca = np.loadtxt(dir_+'/bylayer333K.pca')
SYS5pca = np.loadtxt(dir_+'/bylayer353K.pca')

# whole (merged) set of bylayers
ALLpca = np.loadtxt(dir_+'/all_bylayers.pca')

In [ ]:
# pamm input
default_inputs = dict(
    # cluster
    distance = "minkowski",
    size = 2000,
    p = 2,
    generate_grid = True,
    savegrid = "grid_data",
    # cluster inputs
    d = 5,
    fspread = 0.3,
    ngrid = 2000,
    qs = 1,
    o = "pamm",
    trajectory = dir_+'/all_bylayers.pca',
#     readgrid = "grid_data",
    merger = 0.001,
    bootstrap = 73
)

In [ ]:
datasets_cluster = [
    (ALLpca, {}),
]

datasets_predict = [
    (SYS1pca, {'sys' : '273K', 'nm_frame' : 1152}),
    (SYS2pca, {'sys' : '293K', 'nm_frame' : 1152}),
    (SYS3pca, {'sys' : '323K', 'nm_frame' : 1152}),
    (SYS4pca, {'sys' : '333K', 'nm_frame' : 1152}),
    (SYS5pca, {'sys' : '353K', 'nm_frame' : 1152})
]

## 2) PAMM clustering

In [ ]:
# Clustering on the selected grid of the whole dataset

for i_dataset, (dataset, algo_params) in enumerate(datasets_cluster):
    # update parameters with dataset-specific values
    params = default_inputs.copy()
    params.update(algo_params)

    # Clustering
    p = PammCommander(params)
    print('\n#-----------------------------------------------')
    print(p.command_parser)
    
    print('\nRUNNING Clustering')
    t0 = time.time()
    p.run()
    t1 = time.time()
    print('TIME= '+str(np.round(t1-t0, 2))+' s \n')

In [ ]:
# Prediction on the individuals set and outputs

cluster_output = {}
grid_cluster = {}
prob_output = {}
bootstr_output = {}
systnames = []
for i_dataset,dataset in enumerate(datasets_predict):
    run_syst = str(datasets_predict[i_dataset][1]['sys'])
    # Predict
    print('\nRUNNING Predict '+run_syst)
    t0 = time.time()
    
    gmm = GMMPredict.read_clusters('pamm.pamm', 
                                   grid_file='pamm.grid', 
                                   bootstrap_file='pamm.bs')
        
    print("Reading: pamm.pamm")
    print("There are {} clusters".format(np.unique(gmm.pk).shape[0]))
    x = datasets_predict[i_dataset][0]
    x_ = gmm.predict_proba(x)
    labels = np.argmax(x_, axis=1) #.reshape((-1, 1))

    t1 = time.time()
    print('TIME= '+str(np.round(t1-t0, 2))+' s \n')

    # Storing data
    cluster_output[run_syst] = labels
    grid_cluster[run_syst] = gmm.cluster
    prob_output[run_syst] = gmm.p
    bootstr_output[run_syst] = gmm.bs
    systnames.append(run_syst)
    
    # output for initial clustering
    np.savetxt(run_syst + "_clusters.txt", labels.reshape((-1, 1)))
    
    rates = ClusterRates(datasets_predict[i_dataset][1]['nm_frame'], 'label').calculate_matrix(labels.reshape((-1, 1)))
    np.savetxt(run_syst + "_rates.txt", rates)

## 3) Structural motifs

In [ ]:
# cluster associations

row = 1
col = 1
fig, ax = plt.subplots(row, col, figsize=(col * 5, row * 4), dpi=100)

adjacency, mapping = calculate_adjacency(
prob=prob_output['273K'],
clusters=grid_cluster['273K'],
bootstrap=bootstr_output['273K']
)
    
z = adjancency_dendrogram(adjacency)
_ = dendrogram(z, ax=ax, count_sort=True)['leaves']
    
for k in range(col):
    ax.set_yticks([])
    ax.yaxis.set_ticks_position('none')
    
    for side in ['bottom','right','top','left']:
        ax.spines[side].set_visible(False)


ax.set_ylabel('PAMM DENDROGRAM', size='16')

In [ ]:
# mapping for the macro structural motifs following the cluster dendrogram.
# each entry is a tuple ("name_of_system", dict()) where the dict() contains the cluster associations

mapping = [
    ('273K', {0: [X1,X2,X3],
              1: [Y1,Y2,Y3]}),
    ('293K', {0: [X1,X2,X3],
              1: [Y1,Y2,Y3]}),
    ('323K', {0: [X1,X2,X3],
              1: [Y1,Y2,Y3]}),
    ('333K', {0: [X1,X2,X3],
              1: [Y1,Y2,Y3]}),
    ('353K', {0: [X1,X2,X3],
              1: [Y1,Y2,Y3]}),
]

In [ ]:
# computation of the macro clustering and output of the motifs

macro_cluster_output = {}
rates_macro_clusters = {}

for s,macro_cl in enumerate(systnames):
    # Macro Cluster
    run_syst = macro_cl
    print("MACRO CLUSTER - "+run_syst)
    
    gmm = GMMPredict.read_clusters('pamm.pamm', 
                                   grid_file='pamm.grid', 
                                   bootstrap_file='pamm.bs')
    
    y = datasets_predict[s][0]
    y_ = gmm.predict_proba(y)
    y__ = np.zeros((y.shape[0], len(mapping[s][1])))
    for k, v in mapping[s][1].items():
        y__[:, k] = y_[:,v].sum(1)

    macro_cluster_output[macro_cl] = np.argmax(y__, axis=1)
    np.savetxt(run_syst+'_macro_cluster.dat', np.argmax(y__, axis=1).reshape((-1,1)) )
    
    rates = ClusterRates(datasets_predict[s][1]['nm_frame'], 'label').calculate_matrix(np.argmax(y__, axis=1))
    rates_macro_clusters[macro_cl] = rates
    np.savetxt(run_syst+'_macro_rates.dat', rates)